In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pyro


# Practical exercises with VI

1- Start from the example seen the last week in class. Extend it to estimate a posterior distribution for the variance

In [ ]:
sigma = pyro.sample("sigma", dist.LogNormal(sigma_loc, torch.tensor(0.05)))

2- Consider the following linear regression problem. Find the posterior distribution for the regression parameters

In [ ]:
# Generate some synthetic data
N = 100
x = torch.randn(N, 1)
w_true = 2.0
b_true = 1.0
noise_std = 0.5
y = w_true * x + b_true + noise_std * torch.randn(N, 1)
